In [27]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [28]:

# Function to save data to CSV file
def save_to_csv(data):
    #current_time = datetime.now().strftime("%m-%d-%H-%M-%Y")
    #file_name = f'Verify-sy-V2-{current_time}.csv'
    file_name = 'Verify-sy-3.30PM.csv'
    
    # If the file does not exist, create a new one
    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
           
    
    # Append data to the CSV file
    with open(file_name, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row)


In [29]:
# Function to write log messages to a file
def write_log(message):
    log_file = 'logs.txt'
    with open(log_file, 'a', encoding='utf-8') as file:
        file.write(message + '\n')

In [30]:

# Initialize the WebDriver
driver = webdriver.Chrome()

In [31]:
# Function to get article content and date from new tab
def get_article_info(article_url):
    # Open the article URL in a new tab
    driver.execute_script("window.open('{}', '_blank');".format(article_url))
    # Switch to the new tab
    driver.switch_to.window(driver.window_handles[-1])
    
    # Wait for the article content to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'wekek')))
    
    # Find the element with the class 'wekek'
    article_content_element = driver.find_element(By.CLASS_NAME, 'wekek')
    
    # Extract article content
    article_content = ''
    stop_iteration = False
    for paragraph in article_content_element.find_elements(By.TAG_NAME, 'p'):
        # Skip <p> elements containing images or videos
        if paragraph.find_elements(By.TAG_NAME, 'img') or paragraph.find_elements(By.TAG_NAME, 'video'):
            continue
        article_content += paragraph.text.strip() + '\n'
        
        # Check if we've reached the element with class 'news-details-sections'
        if paragraph.find_elements(By.XPATH, '//h5[@class="news-details-sections"]'):
            stop_iteration = True
            break
    
    # Extract the date from the <p class="dat-det-mob"> element
    date_element = driver.find_element(By.CSS_SELECTOR, 'p.dat-det-mob span')
    article_date = date_element.text.strip()
    
    # Close the new tab
    driver.close()
    # Switch back to the main tab
    driver.switch_to.window(driver.window_handles[0])
    
    return article_content, article_date

In [32]:
""" 
#THIS VERSION WORKED ON THE FIRST LINK ALL PAGES BUT DIDN'T WORK ON THE REST OF THE LINKS CZ OF EXCEPTIONS

# List of URLs to scrape along with their corresponding category names
urls_categories = [
    ('https://verify-sy.com/all/18', 'كذب'),
    ('https://verify-sy.com/all/53', 'تضليل'),
    ('https://verify-sy.com/all/54', 'نظرية-المؤامرة'),
    ('https://verify-sy.com/all/55', 'كذب-باسم-العلم'),
    ('https://verify-sy.com/all/56', 'خطأ'),
    ('https://verify-sy.com/all/57', 'انحياز'),
    ('https://verify-sy.com/all/58', 'تلاعب-بالحقائق'),
    ('https://verify-sy.com/all/59', 'عنوان-مضلل'),
    ('https://verify-sy.com/all/60', 'سخرية'),
    ('https://verify-sy.com/all/61', 'خارج-السياق'),
    ('https://verify-sy.com/all/62', 'غير-مؤكد'),
    ('https://verify-sy.com/all/165', 'مؤكد'),
]

# Initialize the WebDriver
driver = webdriver.Chrome()

try:
    for url, category_name in urls_categories:
        # Open the webpage
        driver.get(url)

        # Give some time for the page to load
        time.sleep(2)

        # Initialize counter for tracking containers
        container_counter = 0

        # Loop to handle "Show more" button
        # Loop to handle "Next page" link
        while True:
            print("Processing page:", driver.current_url)
            # Find and extract the news articles for the current category
            news_articles = driver.find_elements(By.CSS_SELECTOR, 'div.blog_post_style2')
            print("Number of articles found on this page:", len(news_articles))

            # Output the articles for the current category
            data = []
            for article in news_articles:
                title = article.find_element(By.CSS_SELECTOR, 'h3.list-title-ca').text
                article_url = article.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                
                # Get article content and date from new tab
                article_content, article_date = get_article_info(article_url)
                
                # Set the label to 'fake' by default
                label = 'fake'
                # For the URL 'https://verify-sy.com/all/165', set the label to 'real'
                if url == 'https://verify-sy.com/all/165':
                    label = 'real'

                # Append data to the list
                data.append([title, label, category_name, article_content, article_date])

                # Increment the container counter
                container_counter += 1

                # Check if we have processed 18 articles, then break the loop
                #if container_counter >= 18:
                #    break
            
            # Save the data to CSV file
            save_to_csv(data)

            # Check if we have processed 18 articles, then break the loop
            if container_counter >= 18:
            # break

                # Find the "Next page" link
                next_page_link = driver.find_element(By.CSS_SELECTOR, 'link[rel="next"]')
                print("Next page URL:", next_page_link.get_attribute('href'))
                container_counter = 0
            
            # Get the URL of the next page
            next_page_url = next_page_link.get_attribute('href')

            # Open the next page
            driver.get(next_page_url)

            # Wait for new articles to load
            time.sleep(5)  
            WebDriverWait(driver, 10).until(EC.staleness_of(news_articles[-1]))

except Exception as e:
    print("An error occurred:", e)

finally:
    # Close the WebDriver
    driver.quit()

"""

' \n#THIS VERSION WORKED ON THE FIRST LINK ALL PAGES BUT DIDN\'T WORK ON THE REST OF THE LINKS CZ OF EXCEPTIONS\n\n# List of URLs to scrape along with their corresponding category names\nurls_categories = [\n    (\'https://verify-sy.com/all/18\', \'كذب\'),\n    (\'https://verify-sy.com/all/53\', \'تضليل\'),\n    (\'https://verify-sy.com/all/54\', \'نظرية-المؤامرة\'),\n    (\'https://verify-sy.com/all/55\', \'كذب-باسم-العلم\'),\n    (\'https://verify-sy.com/all/56\', \'خطأ\'),\n    (\'https://verify-sy.com/all/57\', \'انحياز\'),\n    (\'https://verify-sy.com/all/58\', \'تلاعب-بالحقائق\'),\n    (\'https://verify-sy.com/all/59\', \'عنوان-مضلل\'),\n    (\'https://verify-sy.com/all/60\', \'سخرية\'),\n    (\'https://verify-sy.com/all/61\', \'خارج-السياق\'),\n    (\'https://verify-sy.com/all/62\', \'غير-مؤكد\'),\n    (\'https://verify-sy.com/all/165\', \'مؤكد\'),\n]\n\n# Initialize the WebDriver\ndriver = webdriver.Chrome()\n\ntry:\n    for url, category_name in urls_categories:\n        # Op

In [33]:
# List of URLs to scrape along with their corresponding category names
urls_categories = [
    ('https://verify-sy.com/all/18', 'كذب'),
    ('https://verify-sy.com/all/53', 'تضليل'),
    ('https://verify-sy.com/all/54', 'نظرية-المؤامرة'),
    ('https://verify-sy.com/all/55', 'كذب-باسم-العلم'),
    ('https://verify-sy.com/all/56', 'خطأ'),
    ('https://verify-sy.com/all/57', 'انحياز'),
    ('https://verify-sy.com/all/58', 'تلاعب-بالحقائق'),
    ('https://verify-sy.com/all/59', 'عنوان-مضلل'),
    ('https://verify-sy.com/all/60', 'سخرية'),
    ('https://verify-sy.com/all/61', 'خارج-السياق'),
    ('https://verify-sy.com/all/62', 'غير-مؤكد'),
    ('https://verify-sy.com/all/165', 'مؤكد'),
]

# Initialize the WebDriver
driver = webdriver.Chrome()

try:
    for url, category_name in urls_categories:
        # Open the webpage
        driver.get(url)
        # Give some time for the page to load
        time.sleep(2)
        # Initialize counter for tracking containers
        container_counter = 0
        # Loop to handle "Next page" link
        while True:
            try:
                log_message = f"Processing page: {driver.current_url}"
                print(log_message)
                write_log(log_message)
                
                # Find and extract the news articles for the current category
                news_articles = driver.find_elements(By.CSS_SELECTOR, 'div.blog_post_style2')
                log_message = f"Number of articles found on this page: {len(news_articles)}"
                write_log(log_message)
                print(log_message)
                # Output the articles for the current category
                data = []
                for article in news_articles:
                    title = article.find_element(By.CSS_SELECTOR, 'h3.list-title-ca').text
                    article_url = article.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                    # Get article content and date from new tab
                    article_content, article_date = get_article_info(article_url)
                    # Set the label to 'fake' by default
                    label = 'fake'
                    # For the URL 'https://verify-sy.com/all/165', set the label to 'real'
                    if url == 'https://verify-sy.com/all/165':
                        label = 'real'
                    # Append data to the list
                    data.append([title, label, category_name, article_content, article_date])
                    # Increment the container counter
                    container_counter += 1
                    # Check if we have processed 18 articles, then break the loop
                    if container_counter >= 18:
                        break
                # Save the data to CSV file
                save_to_csv(data)
                # Check if we have processed 18 articles, then break the loop
                if container_counter >= 18:
                    # Find the "Next page" link
                    next_page_link = driver.find_element(By.CSS_SELECTOR, 'link[rel="next"]')
                    log_message = f"Next page URL: {next_page_link.get_attribute('href')}"
                    write_log(log_message)
                    print(log_message)
                    container_counter = 0
                # Get the URL of the next page
                next_page_url = next_page_link.get_attribute('href')
                # Open the next page
                driver.get(next_page_url)
                # Wait for new articles to load
                time.sleep(5)
                WebDriverWait(driver, 10).until(EC.staleness_of(news_articles[-1]))
            except Exception as e:
                print("An error occurred:", e)
                break  # Break the loop to move to the next category

finally:
    # Close the WebDriver
    driver.quit()

Processing page: https://verify-sy.com/all/18
Number of articles found on this page: 18
Next page URL: https://verify-sy.com/all/18?page=2
Processing page: https://verify-sy.com/all/18?page=2
Number of articles found on this page: 18
Next page URL: https://verify-sy.com/all/18?page=3
Processing page: https://verify-sy.com/all/18?page=3
Number of articles found on this page: 18
Next page URL: https://verify-sy.com/all/18?page=4
Processing page: https://verify-sy.com/all/18?page=4
Number of articles found on this page: 18
Next page URL: https://verify-sy.com/all/18?page=5
Processing page: https://verify-sy.com/all/18?page=5
Number of articles found on this page: 18
Next page URL: https://verify-sy.com/all/18?page=6
Processing page: https://verify-sy.com/all/18?page=6
Number of articles found on this page: 18
Next page URL: https://verify-sy.com/all/18?page=7
Processing page: https://verify-sy.com/all/18?page=7
Number of articles found on this page: 18
Next page URL: https://verify-sy.com